In [ ]:
pip install transformers

In [ ]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset

# Step 1: Download the JSON file
url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
response = requests.get(url)
json_data = response.json()


In [ ]:
def json_to_dataframe(json_data):
    data = []
    for document in json_data:
        doc_id = document.get("id")
        for annotation in document.get("annotations", []):
            for result in annotation.get("result", []):
                segment = {
                    'doc_id': doc_id,
                    'text': result['value'].get('text'),
                    'label': result['value'].get('labels', [None])[0]  # Get the first label if available
                }
                data.append(segment)
    return pd.DataFrame(data)

In [ ]:
# Convert JSON to DataFrame
df = json_to_dataframe(json_data)
print(df.head(100))

    doc_id                                               text     label
0     1735        IN THE HIGH COURT OF KARNATAKA,\n       ...  PREAMBLE
1     1735  \n\n      BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...  PREAMBLE
2     1735  This Criminal Appeal is filed under Section 37...  PREAMBLE
3     1735  \n\n       This appeal coming on for hearing t...  PREAMBLE
4     1735  \n       Heard the learned Counsel for the app...      NONE
..     ...                                                ...       ...
95    4183                                          \nJUSTICE  PREAMBLE
96    4183  .\n\n         THIS I.T.A. COMING ON FOR HEARIN...  PREAMBLE
97    4183          \nS. SUJATHA J. DELIVERED THE FOLLOWING:-  PREAMBLE
98    4183       \n\n                                JUDGMENT  PREAMBLE
99    4183  \nMr. E.I.Sanmathi Adv. for Appellants- Revenu...      NONE

[100 rows x 3 columns]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)
# Convert DataFrame to Hugging Face Dataset again
dataset = Dataset.from_pandas(df)

{'ANALYSIS': 0, 'ARG_PETITIONER': 1, 'ARG_RESPONDENT': 2, 'FAC': 3, 'ISSUE': 4, 'NONE': 5, 'PREAMBLE': 6, 'PRE_NOT_RELIED': 7, 'PRE_RELIED': 8, 'RATIO': 9, 'RLC': 10, 'RPC': 11, 'STA': 12}


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from transformers import AutoModel


In [ ]:
# Load the tokenizer and model
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(df['label'])))

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128) # Added max_length argument

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/28986 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
# Split the dataset into train, test, and eval sets
# Adjust the splits according to your needs
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_test_dataset = split_dataset['test'].train_test_split(test_size=0.5)

datasets = DatasetDict({
    'train': split_dataset['train'],
    'test': train_test_dataset['test'],
    'eval': train_test_dataset['train']
})

In [ ]:

class ContiguousTrainer(Trainer):
    def save_model(self, output_dir=None, _internal_call=True):
        # Ensure that all model weights are contiguous before saving
        for param in self.model.parameters():
            param.data = param.data.contiguous()
        super().save_model(output_dir, _internal_call)
# Step 3: Setup Training Arguments
training_args = TrainingArguments(
    output_dir="./resultsSCIBERT",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_total_limit=1,
    save_steps=500,
    logging_dir="./logs",
    report_to="wandb"  # Ensure W&B is properly set up, or remove this line
)

trainer = ContiguousTrainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['eval']
)
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# If the model is already trained, just initialize the Trainer with the same model
trainer = Trainer(
    model=model,  # Use your already trained model
    #args=training_args,
    eval_dataset=datasets['eval'],  # Use your evaluation dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results
print(results)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


{'eval_loss': 2.5109071731567383, 'eval_model_preparation_time': 0.0064, 'eval_accuracy': 0.09624008278716799, 'eval_precision': 0.21147920496542646, 'eval_recall': 0.09624008278716799, 'eval_f1': 0.09345884454530892, 'eval_runtime': 30.0348, 'eval_samples_per_second': 96.521, 'eval_steps_per_second': 12.086}


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Initialize the Trainer with the already trained model
trainer = Trainer(
    model=model,  # Use your already trained model
    args=training_args,
    eval_dataset=datasets['test'],  # Use your test dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print the evaluation results
print("Test Set Evaluation Results:")
print(results)


Test Set Evaluation Results:
{'eval_loss': 1.0849385261535645, 'eval_model_preparation_time': 0.0038, 'eval_accuracy': 0.661607450845119, 'eval_precision': 0.6541079188692379, 'eval_recall': 0.661607450845119, 'eval_f1': 0.6548580783246429, 'eval_runtime': 18.9801, 'eval_samples_per_second': 152.739, 'eval_steps_per_second': 19.125}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# save this model on huggingface

In [ ]:
pip install huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from huggingface_hub import HfApi

# Define repository name and create repo on the hub
repo_name = "SCIBERT_for_rhetorical_role_labeling"  # Choose a unique name
HfApi().create_repo(repo_name)

# Upload the model
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/engineersaloni159/SCIBERT_for_rhetorical_role_labeling/commit/8f018ed33242292dcf254864c6ff5f1cb9e28751', commit_message='Upload tokenizer', commit_description='', oid='8f018ed33242292dcf254864c6ff5f1cb9e28751', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained(repo_name, push_to_hub=True)
tokenizer.save_pretrained(repo_name, push_to_hub=True)

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


('SCIBERT_for_rhetorical_role_labeling/tokenizer_config.json',
 'SCIBERT_for_rhetorical_role_labeling/special_tokens_map.json',
 'SCIBERT_for_rhetorical_role_labeling/vocab.txt',
 'SCIBERT_for_rhetorical_role_labeling/added_tokens.json',
 'SCIBERT_for_rhetorical_role_labeling/tokenizer.json')

# Manually Evaluation of model

In [ ]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
Found existing installation: torchvision 0.19.0+cu121
Uninstalling torchvision-0.19.0+cu121:
  Successfully uninstalled torchvision-0.19.0+cu121
Found existing installation: torchaudio 2.4.0+cu121
Uninstalling torchaudio-2.4.0+cu121:
  Successfully uninstalled torchaudio-2.4.0+cu121


In [ ]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import load_dataset, Dataset
import pandas as pd
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("engineersaloni159/SCIBERT_for_rhetorical_role_labeling")
model = AutoModelForSequenceClassification.from_pretrained("engineersaloni159/SCIBERT_for_rhetorical_role_labeling")


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/717k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Ensure model is in evaluation mode
model.eval()

In [ ]:
#run this cell if model is directly called from  huggingface
from sklearn.preprocessing import LabelEncoder

# Assuming you already have this
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Print the mapping of labels to numbers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

NameError: name 'df' is not defined

In [ ]:
# Assuming label_encoder is the one you used for encoding labels earlier

# Example sentence
sentence =  "Civil Appeal No. 40 of 1960 must, therefore, be allowed."
# Tokenize the sentence
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

# The predictions are in the form of logits, you can convert them to probabilities
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1)

# Convert the predicted class number to the original label
predicted_label = label_encoder.inverse_transform(predicted_class.cpu().numpy())[0]

# Print the predicted label
print(f"Predicted label: {predicted_label}")


Predicted label: RPC


In [ ]:
print("Logits:", logits)


Logits: tensor([[ 2.2846,  0.2215, -0.3713, -0.5981, -0.7258, -0.8616,  0.2079, -1.1513,
          0.6050, -0.1885, -0.6332, -0.5202, -0.2322]])
